In [1]:
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

%pip install sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:        11536372 kB
MemAvailable:   12369648 kB


**학과·학번별 베스트 대출 TOP10을 추후 생성할 tfidf_matrix에 포함시키기 위해 데이터프레임 생성**

단행본_대출, 단행본_도서 불러오기

In [4]:
# 단행본_대출
file_lib_bor = '/content/drive/MyDrive/datathon/데이터_단행본대출.csv'
lib_bor = pd.read_csv(file_lib_bor, encoding = 'cp949',low_memory=False)
lib_bor = lib_bor.dropna(how='any') # 결측치 제거
lib_bor= lib_bor.drop([lib_bor.columns[0],lib_bor.columns[1],lib_bor.columns[2],lib_bor.columns[5],lib_bor.columns[9]], axis= 1)
# 단행본_도서
file_lib_book = '/content/drive/MyDrive/datathon/데이터_단행본도서.csv'
lib_book = pd.read_csv(file_lib_book, encoding = 'cp949',low_memory=False)
lib_book = lib_book.dropna(how='any') # 결측치 제거
lib_book = lib_book.drop([lib_book.columns[0],lib_book.columns[1],lib_book.columns[2],lib_book.columns[3],lib_book.columns[6],lib_book.columns[7],lib_book.columns[8],lib_book.columns[9],lib_book.columns[11],lib_book.columns[12]], axis = 1)

데이터 병합

In [5]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:         9762160 kB
MemAvailable:   12031000 kB


In [6]:
bor_book =pd.merge(lib_bor,lib_book, on = ['ISBN','서명'], how = 'outer')
bor_book = bor_book.dropna(how='any') # 결측치 제거
bor_book.index = np.arange(len(bor_book))
bor_book['연대출권수'] = bor_book['연대출권수'].astype(int)
bor_book['입학년도'] = bor_book['입학년도'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


문제점 : 각각의 학생이 여러권의 도서를 빌릴 경우 맨 앞의 서명만이 남음

해결책 : primary key에 서명을 추가

In [7]:
bor_book =bor_book.drop_duplicates(['연대출권수' ,'입학년도','소속','서명'])
bor_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59428 entries, 0 to 377011
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    59428 non-null  object
 1   서명      59428 non-null  object
 2   연대출권수   59428 non-null  int64 
 3   입학년도    59428 non-null  int64 
 4   소속      59428 non-null  object
 5   분류코드    59428 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.2+ MB


In [8]:
bor_book.head(10)

,ISBN,서명,연대출권수,입학년도,소속,분류코드
0,9788970551234,소유냐 삶이냐,90,2014,금융경제학과,111.1
2,9788970551234,소유냐 삶이냐,6,2016,컴퓨터학부,111.1
6,9788970551234,소유냐 삶이냐,7,2014,컴퓨터학부,111.1
8,9788970551234,소유냐 삶이냐,95,2010,경제학과,111.1
16,9788970551234,소유냐 삶이냐,39,2017,금융학부,111.1
20,9788970551234,소유냐 삶이냐,195,2012,글로벌미디어학부,111.1
22,9788970551234,소유냐 삶이냐,85,2017,철학과,111.1
26,9788970551234,소유냐 삶이냐,26,2017,철학과,111.1
28,9791186289501,노동법 원포인트 판례 170선,49,2010,벤처중소기업학과,348.6
30,9791186289501,노동법 원포인트 판례 170선,46,2010,벤처중소기업학과,348.6


In [9]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:         9779268 kB
MemAvailable:   12055200 kB


학과&학번별 dictionary key 생성

In [10]:
tmp_group = bor_book.groupby(['소속','입학년도'])

In [11]:
tmp_dic = dict(list(tmp_group))
tmp_dic.keys()

dict_keys([('AI융합학부', 2011), ('AI융합학부', 2012), ('AI융합학부', 2013), ('AI융합학부', 2014), ('AI융합학부', 2015), ('AI융합학부', 2016), ('AI융합학부', 2017), ('AI융합학부', 2018), ('AI융합학부', 2019), ('AI융합학부', 2020), ('AI융합학부', 2021), ('건축학부', 1995), ('건축학부', 1999), ('건축학부', 2000), ('건축학부', 2003), ('건축학부', 2005), ('건축학부', 2007), ('건축학부', 2008), ('건축학부', 2009), ('건축학부', 2010), ('건축학부', 2011), ('건축학부', 2012), ('건축학부', 2013), ('건축학부', 2014), ('건축학부', 2015), ('건축학부', 2016), ('건축학부', 2017), ('건축학부', 2018), ('건축학부', 2019), ('건축학부', 2020), ('건축학부', 2021), ('경영학부', 2006), ('경영학부', 2007), ('경영학부', 2008), ('경영학부', 2009), ('경영학부', 2010), ('경영학부', 2011), ('경영학부', 2012), ('경영학부', 2013), ('경영학부', 2014), ('경영학부', 2015), ('경영학부', 2016), ('경영학부', 2017), ('경영학부', 2018), ('경영학부', 2019), ('경영학부', 2020), ('경영학부', 2021), ('경제학과', 1999), ('경제학과', 2006), ('경제학과', 2008), ('경제학과', 2009), ('경제학과', 2010), ('경제학과', 2011), ('경제학과', 2012), ('경제학과', 2013), ('경제학과', 2014), ('경제학과', 2015), ('경제학과', 2016), ('경제학과', 2017), ('경제학과', 2018), ('경제학과'

In [12]:
exist_year = []
for n in range (2012,2022):
   exist_year.append(n)
exist_year

[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021]

In [13]:
tmp_lst = list(bor_book['소속'])
exist_dept = list(set(tmp_lst)) #중복원소 제거 단 순서는 뒤죽박죽 -> 무관하기에 사용
print(exist_dept)

['철학과', '금융경제학과', '정치외교학과', '소프트웨어학부', '전자정보공학부 IT융합전공', '예술창작학부 영화예술전공', '기독교학과', '평생교육학과', '경제학과', '사학과', '국어국문학과', '불어불문학과', '예술창작학부 문예창작전공', '경영학부', '산업·정보시스템공학과', '중어중문학과', '금융학부', '융합특성화자유전공학부', '회계학과', '언론홍보학과', '복지경영학과(계약학과)', '컴퓨터학부', '독어독문학과', '물리학과', '벤처경영학과(계약학과)', '일어일문학과', '법학과', '화학공학과', '전자정보공학부 전자공학전공', '정보통신전자공학부', '유기신소재·파이버공학과', '국제법무학과', '벤처중소기업학과', '화학과', '글로벌통상학과', '행정학부', 'AI융합학부', '국제무역학과', '스포츠학부', '수학과', '건축학부', '전기공학부', '미디어경영학과', '정보사회학과', '영어영문학과', '기계공학부', '글로벌미디어학부', '정보통계·보험수리학과', '의생명시스템학부', '혁신경영학과(계약학과)', '사회복지학부']


In [14]:
#12~ 21
best_seller_dataframe = []
for year in exist_year:
   for dept in exist_dept:
               
       if (year < 2012):
           continue
       else:
           tmp_data_frame = bor_book[bor_book['입학년도']== year]
           tmp_data_frame = tmp_data_frame[tmp_data_frame['소속']== dept]
           best_seller_dataframe.append(pd.DataFrame(data= tmp_data_frame))

In [15]:
import math
#학번 별 베스트 셀러(2012~2021)
best = []
for dept, year in tmp_dic.keys(): #unpacking
    target_dept = ''

    for index_num in range(0,len(best_seller_dataframe)):
        my_zip = best_seller_dataframe[index_num]['소속'].value_counts().iteritems()
        for tmp_target_dept,count in best_seller_dataframe[index_num]['소속'].value_counts().iteritems(): #언패킹하기
            target_dept = tmp_target_dept
            
        if math.isnan(best_seller_dataframe[index_num]['입학년도'].mean()):
            continue
    
        if int(best_seller_dataframe[index_num]['입학년도'].mean()) == year and target_dept == dept: #보고 싶은 학번, 학과
            print("\n-------------------------",dept,"    ",year,"-------------------------")
            test = best_seller_dataframe[index_num]["서명"]
            item_count = test.value_counts()[:10]
            for idx, (val,cnt) in enumerate (item_count.iteritems(),1):
                #print("")
                #print('Top',idx,":",val)
                best.append(val)


------------------------- AI융합학부      2012 -------------------------

------------------------- AI융합학부      2013 -------------------------

------------------------- AI융합학부      2014 -------------------------

------------------------- AI융합학부      2015 -------------------------

------------------------- AI융합학부      2016 -------------------------

------------------------- AI융합학부      2017 -------------------------

------------------------- AI융합학부      2018 -------------------------

------------------------- AI융합학부      2019 -------------------------

------------------------- AI융합학부      2020 -------------------------

------------------------- AI융합학부      2021 -------------------------

------------------------- 건축학부      2012 -------------------------

------------------------- 건축학부      2013 -------------------------

------------------------- 건축학부      2014 -------------------------

------------------------- 건축학부      2015 -------------------------

------------------------- 건

중복제거

In [16]:
my_set = set(best) #집합set으로 변환
best = list(my_set)
len(best)

2336

In [17]:
type(lib_book["ISBN"][11])

str

**best_top10** => 학과·학번별 베스트 대출 TOP10 데이터 프레임 생성

In [18]:
Nan_list = [('Na','Na','Na')]
best_top10 = pd.DataFrame(Nan_list, columns = ['ISBN' , '서명', '분류코드'])
for name in best:
    l_idx = lib_book.loc[lib_book["서명"]==name].index
    idx = l_idx[0]
    best_top10 = best_top10.append({'ISBN' : lib_book["ISBN"][idx] , 
                  '서명' : lib_book["서명"][idx] , '분류코드' : lib_book["분류코드"][idx]} , ignore_index=True)
best_top10.drop(labels=0,axis=0)

,ISBN,서명,분류코드
1,9780735590427,Evidence,347.7306
2,9781380003089,Breakthrough plus :student's book.v.1,428.24
3,9788991656871,전기기기,621.31042
4,9788964143520,투자분쟁 해결을 위한 국제중심 협정 :규정 및 규칙 해설,341.3
5,9788972831600,(A) book on C: programming in C,005.133C1
...,...,...,...
2332,9788970960593,獨逸文學思潮史,830.9
2333,9791156643951,C 언어 for beginner,005.133
2334,9788971156452,희곡 창작의 실제,808.2
2335,9780133925852,Teaching by principles: an interactive approac...,401.80071


In [19]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:         9699520 kB
MemAvailable:   11988200 kB



# ***도서별 상세 정보(문화데이터)***


문화데이터 불러오기 2019 - 202112

원본데이터의 손실을 방지하기위해 하나씩 불러와서 합치자

***2019***

In [20]:
file_book_2019 = '/content/drive/MyDrive/datathon/NL_bo_species_master_new_2019.csv'
book_2019 = pd.read_csv(file_book_2019, low_memory = False, skiprows = [783400, 2002735])
book_2019 = book_2019.drop(['?master_seq', 'vol', 'author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url'], axis = 1)
book_2019 = book_2019.drop_duplicates(['description'])
book_2019.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1199652 entries, 0 to 3178394
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   isbn13       1199648 non-null  object
 1   title        1199650 non-null  object
 2   description  1199651 non-null  object
dtypes: object(3)
memory usage: 36.6+ MB


***202012-1***

In [21]:
file_book_202012_1 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202012-1.csv'
book_202012_1 = pd.read_csv(file_book_202012_1, low_memory = False)
book_202012_1 = book_202012_1.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                  'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202012_1 = book_202012_1.drop_duplicates(['description'])
book_202012_1.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 479061 entries, 0 to 999427
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn13         479061 non-null  object
 1   title          479061 non-null  object
 2   description    479060 non-null  object
 3   title_replace  479057 non-null  object
dtypes: object(4)
memory usage: 18.3+ MB


***202012-2***

In [22]:
file_book_202012_2 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202012-2.csv'
book_202012_2 = pd.read_csv(file_book_202012_2, low_memory = False)
book_202012_2 = book_202012_2.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                  'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202012_2 = book_202012_2.drop_duplicates(['description'])
book_202012_2.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420753 entries, 0 to 991323
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn13         420753 non-null  object
 1   title          420753 non-null  object
 2   description    420752 non-null  object
 3   title_replace  330670 non-null  object
dtypes: object(4)
memory usage: 16.1+ MB


***202012-3***

In [23]:
file_book_202012_3 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202012-3.csv'
book_202012_3 = pd.read_csv(file_book_202012_3, low_memory = False)
book_202012_3 = book_202012_3.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                  'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202012_3 = book_202012_3.drop_duplicates(['description'])
book_202012_3.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 208322 entries, 0 to 990346
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn13         208322 non-null  object
 1   title          208322 non-null  object
 2   description    208321 non-null  object
 3   title_replace  200524 non-null  object
dtypes: object(4)
memory usage: 7.9+ MB


***202012-4***

In [24]:
file_book_202012_4 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202012-4.csv'
book_202012_4 = pd.read_csv(file_book_202012_4, low_memory = False)
book_202012_4 = book_202012_4.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                  'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202012_4 = book_202012_4.drop_duplicates(['description'])
book_202012_4.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186588 entries, 0 to 464262
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn13         186588 non-null  int64 
 1   title          186588 non-null  object
 2   description    186587 non-null  object
 3   title_replace  184466 non-null  object
dtypes: int64(1), object(3)
memory usage: 7.1+ MB


***202012-5***

In [25]:
file_book_202012_5 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202012-5.csv'
book_202012_5 = pd.read_csv(file_book_202012_5, low_memory = False)
book_202012_5 = book_202012_5.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                  'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202012_5 = book_202012_5.drop_duplicates(['description'])
book_202012_5.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11789 entries, 0 to 37582
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         11789 non-null  int64 
 1   title          11789 non-null  object
 2   description    11788 non-null  object
 3   title_replace  11705 non-null  object
dtypes: int64(1), object(3)
memory usage: 460.5+ KB


***202101***

In [26]:
file_book_202101 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202101.csv'
book_202101 = pd.read_csv(file_book_202101, low_memory = False)
book_202101 = book_202101.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202101 = book_202101.drop_duplicates(['description'])
book_202101.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4629 entries, 0 to 11050
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         4629 non-null   int64 
 1   title          4629 non-null   object
 2   description    4628 non-null   object
 3   title_replace  4501 non-null   object
dtypes: int64(1), object(3)
memory usage: 180.8+ KB


***202102***

In [27]:
file_book_202102 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202102.csv'
book_202102 = pd.read_csv(file_book_202102, low_memory = False)
book_202102 = book_202102.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202102 = book_202102.drop_duplicates(['description'])
book_202102.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165733 entries, 0 to 448354
Data columns (total 4 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   isbn13         165733 non-null  int64 
 1   title          165733 non-null  object
 2   description    165732 non-null  object
 3   title_replace  165581 non-null  object
dtypes: int64(1), object(3)
memory usage: 6.3+ MB


In [28]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:          741532 kB
MemAvailable:   10986120 kB


***202103***

In [29]:
file_book_202103 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202103.csv'
book_202103 = pd.read_csv(file_book_202103, low_memory = False)
book_202103 = book_202103.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202103 = book_202103.drop_duplicates(['description'])
book_202103.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9063 entries, 0 to 26666
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         9063 non-null   int64 
 1   title          9063 non-null   object
 2   description    9062 non-null   object
 3   title_replace  8902 non-null   object
dtypes: int64(1), object(3)
memory usage: 354.0+ KB


***202104***

In [30]:
file_book_202104 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202104.csv'
book_202104 = pd.read_csv(file_book_202104, low_memory = False)
book_202104 = book_202104.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202104 = book_202104.drop_duplicates(['description'])
book_202104.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16481 entries, 0 to 33433
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         16481 non-null  int64 
 1   title          16481 non-null  object
 2   description    16480 non-null  object
 3   title_replace  16016 non-null  object
dtypes: int64(1), object(3)
memory usage: 643.8+ KB


***202105***

In [31]:
file_book_202105 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202105.csv'
book_202105 = pd.read_csv(file_book_202105, low_memory = False)
book_202105 = book_202105.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202105 = book_202105.drop_duplicates(['description'])
book_202105.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15631 entries, 0 to 34328
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         15631 non-null  int64 
 1   title          15631 non-null  object
 2   description    15630 non-null  object
 3   title_replace  15112 non-null  object
dtypes: int64(1), object(3)
memory usage: 610.6+ KB


***202106***

In [32]:
file_book_202106 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202106.csv'
book_202106 = pd.read_csv(file_book_202106, low_memory = False)
book_202106 = book_202106.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202106 = book_202106.drop_duplicates(['description'])
book_202106.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24776 entries, 0 to 71652
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         24776 non-null  int64 
 1   title          24776 non-null  object
 2   description    24775 non-null  object
 3   title_replace  24120 non-null  object
dtypes: int64(1), object(3)
memory usage: 967.8+ KB


***202107***

In [33]:
file_book_202107 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202107.csv'
book_202107 = pd.read_csv(file_book_202107, low_memory = False)
book_202107 = book_202107.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202107 = book_202107.drop_duplicates(['description'])
book_202107.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7292 entries, 0 to 23797
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         7292 non-null   int64 
 1   title          7292 non-null   object
 2   description    7291 non-null   object
 3   title_replace  6669 non-null   object
dtypes: int64(1), object(3)
memory usage: 284.8+ KB


***202108***

In [34]:
file_book_202108 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202108.csv'
book_202108 = pd.read_csv(file_book_202108, low_memory = False)
book_202108 = book_202108.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202108 = book_202108.drop_duplicates(['description'])
book_202108.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31817 entries, 0 to 64246
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         31817 non-null  int64 
 1   title          31817 non-null  object
 2   description    31816 non-null  object
 3   title_replace  31330 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.2+ MB


***202109***

In [35]:
file_book_202109 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202109.csv'
book_202109 = pd.read_csv(file_book_202109, low_memory = False)
book_202109 = book_202109.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202109 = book_202109.drop_duplicates(['description'])
book_202109.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29966 entries, 0 to 267579
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         29966 non-null  int64 
 1   title          29966 non-null  object
 2   description    29965 non-null  object
 3   title_replace  29130 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.1+ MB


***202110***

In [36]:
file_book_202110 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202110.csv'
book_202110 = pd.read_csv(file_book_202110, low_memory = False)
book_202110 = book_202110.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202110 = book_202110.drop_duplicates(['description'])
book_202110.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6875 entries, 0 to 16836
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         6875 non-null   int64 
 1   title          6875 non-null   object
 2   description    6874 non-null   object
 3   title_replace  6329 non-null   object
dtypes: int64(1), object(3)
memory usage: 268.6+ KB


***202111***

In [37]:
file_book_202111 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202111.csv'
book_202111 = pd.read_csv(file_book_202111, low_memory = False)
book_202111 = book_202111.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202111 = book_202111.drop_duplicates(['description'])
book_202111.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46311 entries, 0 to 102818
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         46311 non-null  int64 
 1   title          46311 non-null  object
 2   description    46310 non-null  object
 3   title_replace  45465 non-null  object
dtypes: int64(1), object(3)
memory usage: 1.8+ MB


***202112***

In [38]:
file_book_202112 = '/content/drive/MyDrive/datathon/NL_BO_SPECIES_MASTER_NEW_202112.csv'
book_202112 = pd.read_csv(file_book_202112, low_memory = False)
book_202112 = book_202112.drop(['author', 'publisher', 'pub_date', 'add_code', 'price', 'img_url', 'kdc_class_no', 
                 'master_seq', 'vol', 'isbn_origin', 'author_replace', 'pub_date_2', 'is_coll_aladin', 'is_coll_naver'], axis = 1)
book_202112 = book_202112.drop_duplicates(['description'])
book_202112.info()
# 문화데이터 불러와서 description에 중복되는 내용 및 결측치 제거

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74338 entries, 0 to 157111
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   isbn13         74338 non-null  int64 
 1   title          74338 non-null  object
 2   description    74337 non-null  object
 3   title_replace  73119 non-null  object
dtypes: int64(1), object(3)
memory usage: 2.8+ MB


In [39]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:          160928 kB
MemAvailable:   10972932 kB


# ***불러온 문화데이터 합치기***
모든 문화데이터를 concat()한 book을 생성

In [120]:
book = pd.concat([book_2019, book_202012_1, book_202012_2, book_202012_3, book_202012_4, book_202012_5, book_202101, book_202102,
                  book_202103, book_202104, book_202105, book_202106, book_202107, book_202108, book_202109, book_202110, 
                  book_202111, book_202112])
book = book.drop_duplicates(['description'])
book = book.drop_duplicates(['title']) 
book = book.drop_duplicates(['isbn13']) 
# 중복되는 row 삭제
book = book.dropna(subset = ['description'])
book = book.dropna(subset = ['title'])
book = book.dropna(subset = ['isbn13'])
# NAN값 제거
book = book.reset_index(drop=True)
# index 오름차순 재배열
book = book.drop(['title_replace'], axis = 1)
# 불필요해 보이는 title_replace 삭제
book.rename(columns = {'isbn13' : 'ISBN'}, inplace = True)
# isbn13 -> ISBN
book

,ISBN,title,description
0,9788953278974,신곡주계 폴리포니카,신곡 연주를 듣고 힘을 발휘하는 정령과 그 정령에게 힘을 주는 신곡 악사의 이야기....
1,9788953283589,아수라 크라잉 =Asura cryin',라쿠로와 고등학교에서 도난 사건이 발생했다.&#x0D;범인은「하멜른의 속옷남」이라는...
2,9788974572549,(English) fantasy,"국내에서 기획 제작한 어린이용 영어 팬터지. 간결한 문장, 사전을 찾지 않아도 될 ..."
3,9788926325117,제로의 사역마,"'역시 난 사이토를 좋아하나 봐.' '보이는 거, 그렇게 싫지 않았어.' '난 몹쓸..."
4,9788953283459,디어 왈츠 =Dear waltz,투덜이 미소년 강세겸은 요즘 헷갈려하는 중이다. 미향이를 보며 가슴이 뛰는 건 연애...
...,...,...,...
1244545,9791197569708,우리 없는 세상,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...
1244546,9791138804486,위대한 영혼의 미소,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...
1244547,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...
1244548,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...


In [121]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244550 entries, 0 to 1244549
Data columns (total 3 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ISBN         1244550 non-null  object
 1   title        1244550 non-null  object
 2   description  1244550 non-null  object
dtypes: object(3)
memory usage: 28.5+ MB


title과 description을 하나의 컬럼으로 합친 새로운 column 'overview' 생성.. 

컨텐츠(줄거리) 기반의 필터링 실시에 있어서 정확도 향상을 위하여(주요 키워드 빈도수 향상을 위해)

In [122]:
cols = ['title', 'description']
book['overview'] = book[cols].apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)
# title과 description을 합치는데에 있어서 TfidfVectorizer을 위해 한 칸 띄어서 합치기
book

,ISBN,title,description,overview
0,9788953278974,신곡주계 폴리포니카,신곡 연주를 듣고 힘을 발휘하는 정령과 그 정령에게 힘을 주는 신곡 악사의 이야기....,신곡주계 폴리포니카 신곡 연주를 듣고 힘을 발휘하는 정령과 그 정령에게 힘을 주는 ...
1,9788953283589,아수라 크라잉 =Asura cryin',라쿠로와 고등학교에서 도난 사건이 발생했다.&#x0D;범인은「하멜른의 속옷남」이라는...,아수라 크라잉 =Asura cryin' 라쿠로와 고등학교에서 도난 사건이 발생했다....
2,9788974572549,(English) fantasy,"국내에서 기획 제작한 어린이용 영어 팬터지. 간결한 문장, 사전을 찾지 않아도 될 ...",(English) fantasy 국내에서 기획 제작한 어린이용 영어 팬터지. 간결한...
3,9788926325117,제로의 사역마,"'역시 난 사이토를 좋아하나 봐.' '보이는 거, 그렇게 싫지 않았어.' '난 몹쓸...","제로의 사역마 '역시 난 사이토를 좋아하나 봐.' '보이는 거, 그렇게 싫지 않았..."
4,9788953283459,디어 왈츠 =Dear waltz,투덜이 미소년 강세겸은 요즘 헷갈려하는 중이다. 미향이를 보며 가슴이 뛰는 건 연애...,디어 왈츠 =Dear waltz 투덜이 미소년 강세겸은 요즘 헷갈려하는 중이다. 미...
...,...,...,...,...
1244545,9791197569708,우리 없는 세상,작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃었거나 스스로 자...,우리 없는 세상 작가는 사회복지사와 상담가로 오래 일했다. 그는 자살로 누군가를 잃...
1244546,9791138804486,위대한 영혼의 미소,재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 이민 역정을 다룬 이...,위대한 영혼의 미소 재외 작가의 장편소설. 인고와 눈물로 점철된 코리안 뉴요커들의 ...
1244547,9791138804523,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리",갑작스레 세상에 나 혼자 남겨진 느낌이 들 때가 있다. 주위가 암전되고 덩그러니 남...,"잊지 말자, 나는 기적이라는 걸 - 늘 그랬던 것처럼, 신발 끈을 고쳐 맬 우리 갑..."
1244548,9791197549335,전념 - 나와 세상을 바꾸는 힘에 관하여,오랫동안 하나의 목적을 이루기 위해 전념한 다양한 영웅들의 사례도 제시하고 있다. ...,전념 - 나와 세상을 바꾸는 힘에 관하여 오랫동안 하나의 목적을 이루기 위해 전념한...


In [123]:
book['overview'].head()

0    신곡주계 폴리포니카 신곡 연주를 듣고 힘을 발휘하는 정령과 그 정령에게 힘을 주는 ...
1    아수라 크라잉 =Asura cryin' 라쿠로와 고등학교에서 도난 사건이 발생했다....
2    (English) fantasy 국내에서 기획 제작한 어린이용 영어 팬터지. 간결한...
3    제로의 사역마  '역시 난 사이토를 좋아하나 봐.' '보이는 거, 그렇게 싫지 않았...
4    디어 왈츠 =Dear waltz 투덜이 미소년 강세겸은 요즘 헷갈려하는 중이다. 미...
Name: overview, dtype: object

In [124]:
book['overview'].isnull().values.any()
# description column에 결측치 없는 것 확인
# 앞의 전처리과정에서 결측지 모두 제거함

False

**1. overview의 영어와 한글을 제외한 특수문자 및 숫자등은 모두 삭제**

In [125]:
import re
book['overview'] = [re.sub(r'\d+', '', s) for s in book['overview']]
book['overview'] = [re.sub(r'[^\w\s]', '', s) for s in book['overview']]
book['overview'].tail()

1244545    우리 없는 세상 작가는 사회복지사와 상담가로 오래 일했다 그는 자살로 누군가를 잃었...
1244546    위대한 영혼의 미소 재외 작가의 장편소설 인고와 눈물로 점철된 코리안 뉴요커들의 이...
1244547    잊지 말자 나는 기적이라는 걸  늘 그랬던 것처럼 신발 끈을 고쳐 맬 우리 갑작스레...
1244548    전념  나와 세상을 바꾸는 힘에 관하여 오랫동안 하나의 목적을 이루기 위해 전념한 ...
1244549     예산회계실무 기본서  지방자치단체 및 공공기관의 지방자치단체와 공공기관공기업 출자...
Name: overview, dtype: object

## **======================================================**
## **======================================================**


# (번외)
 단행본_도서와 문화데이터에 공통적으로 존재하는 (ISBN을 기준으로) 도서의 데이터프레임

***타당성을 주장하기위해선 TOP10과 관련하여 중복율이 꽤 된다는것을 입증해야한다. ***

In [126]:
merge_book = pd.merge(lib_book, book, on = ['ISBN'], how = 'inner')
merge_book = merge_book.drop_duplicates(['ISBN'])
merge_book = merge_book.drop(['분류코드', '서명'], axis = 1)
merge_book.head()

,ISBN,title,description,overview
0,9788988383216,산업안전보건법·영·규칙 연구실 안전환경 조성에 관한 법률 업무편람,"산업안전보건법 편람집. 이 책은 산업안전보건법과 령, 규칙, 연구실 안전환경 조성에...",산업안전보건법영규칙 연구실 안전환경 조성에 관한 법률 업무편람 산업안전보건법 편람집...
2,9780345803504,Fifty Shades Freed (Paperback) - Fifty Shades ...,When unworldly student Ana Steele first encoun...,Fifty Shades Freed Paperback Fifty Shades Tri...
3,9788961670968,(존 스타인벡의) 진주,1940년 &lt;분노의 포도&gt;로 퓰리처상을 수상한 후 부와 명성을 얻은 스타...,존 스타인벡의 진주 년 lt분노의 포도gt로 퓰리처상을 수상한 후 부와 명성을 얻...
5,9788959944002,파리의 어떤 하루,파리는 도시 전체가 마치 고전과 현대를 넘나드는 스펙터클한 한 편의 영화와 같다. ...,파리의 어떤 하루 파리는 도시 전체가 마치 고전과 현대를 넘나드는 스펙터클한 한 ...
7,9788993566086,와인 테이스팅의 이해 =Michael schuster의 와인 데이스팅 기술에 관한 ...,세계에서 가장 섬세한 와인 교육가가 전해주는 상세하고도 전문적인 테이스팅 강의『와인...,와인 테이스팅의 이해 Michael schuster의 와인 데이스팅 기술에 관한 모...


In [127]:
merge_book.info()
# 총 20249권이 겹침

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20249 entries, 0 to 35282
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         20249 non-null  object
 1   title        20249 non-null  object
 2   description  20249 non-null  object
 3   overview     20249 non-null  object
dtypes: object(4)
memory usage: 791.0+ KB


겹치는 책 일일이 찾아보기

In [128]:
# a = merge_book[(merge_book['서명'] == '넛지: 똑똑한 선택을 이끄는 힘')]
# a
# 철학과 2017 -> 이방인, 중국철학사 빼고 다 있음 -> 8/10
# 컴학 2019 -> 이산수학, 네트워크 쉽게, 안드로이드 앱 플밍, concepts~ 빼고 다 있음 -> 6/10
# 국문 2020 -> 한국대표설화2 빼고 다 있음 -> 9/10

**(국어국문학과&2020)**중복율 90%로 높은편이므로 이 데이터프레임을 가지고 코사인유사도를 구하자

*(국어국문학과&2020) TOP10 데이터프레임 생성하기*

In [129]:
# DKLL_2020 = merge_book[(merge_book['서명']=='한국 구비문학의 이해') | (merge_book['서명']=='한국대표설화1 :신성과 질서의 세계') | (merge_book['서명']=='국어문법론강의')
#  | (merge_book['서명']=='백(百)의 그림자  :황정은 장편소설') | (merge_book['서명']=='한국대표설화2 :위안과 평등의 길') | (merge_book['서명']=='넛지: 똑똑한 선택을 이끄는 힘')
#   | (merge_book['서명']=='한국어 형태론') | (merge_book['서명']=='표준 국어문법론') | (merge_book['서명']=='알기 쉬운 양자론: 현대물리학을 만든 거인들')
#    | (merge_book['서명']=='현대경영의 이해')]
# # (국어국문학과&2020) TOP10 데이터프레임 생성
# DKLL_2020 = DKLL_2020.drop_duplicates(['서명'])
# # 서명을 기준으로 중복 데이터 삭제

# DKLL_2020

## **======================================================**
## **======================================================**


**2. 영어와 한글 불용어 사전을 이용해서 불필요한 조사 및 관사 삭제**

영어 불용어 사전

In [130]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_eng = TfidfVectorizer(stop_words = 'english')
# 영어의 경우 불용어사전이 scikit-learn 라이브러리에 있으므로 이용하자 (한글의 경우 없으므로 만들어줘야됨)
# tfidf_eng = 영어의 불필요한 관사등을 제거하기 위한 객체

In [131]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
ENGLISH_STOP_WORDS
# 영어 불용어 사전에는 어떤게 있는지

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

한글 불용어 사전

In [132]:
korean_nondic = '/content/drive/MyDrive/datathon/한글 불용어 사전.txt'
with open(korean_nondic, 'r', encoding = "utf-8") as f:
  list_file = f.readlines() 
stopwords_kor = [l.strip() for l in list_file]
# stopwords_kor = list_file[0].split(",")
# 한글 불용어들을 작성한 텍스트파일(이하 한글 불용어 사전) 불러오기

In [133]:
stopwords_kor
# 한국 불용어 사전에는 어떤게 있는지

['아',
 '휴',
 '아이구',
 '아이쿠',
 '아이고',
 '어',
 '나',
 '우리',
 '저희',
 '따라',
 '의해',
 '을',
 '를',
 '에',
 '의',
 '가',
 '으로',
 '로',
 '에게',
 '뿐이다',
 '의거하여',
 '근거하여',
 '입각하여',
 '기준으로',
 '예하면',
 '예를 들면',
 '예를 들자면',
 '저',
 '소인',
 '소생',
 '저희',
 '지말고',
 '하지마',
 '하지마라',
 '다른',
 '물론',
 '또한',
 '그리고',
 '비길수 없다',
 '해서는 안된다',
 '뿐만 아니라',
 '만이 아니다',
 '만은 아니다',
 '막론하고',
 '관계없이',
 '그치지 않다',
 '그러나',
 '그런데',
 '하지만',
 '든간에',
 '논하지 않다',
 '따지지 않다',
 '설사',
 '비록',
 '더라도',
 '아니면',
 '만 못하다',
 '하는 편이 낫다',
 '불문하고',
 '향하여',
 '향해서',
 '향하다',
 '쪽으로',
 '틈타',
 '이용하여',
 '타다',
 '오르다',
 '제외하고',
 '이 외에',
 '이 밖에',
 '하여야',
 '비로소',
 '한다면 몰라도',
 '외에도',
 '이곳',
 '여기',
 '부터',
 '기점으로',
 '따라서',
 '할 생각이다',
 '하려고하다',
 '이리하여',
 '그리하여',
 '그렇게 함으로써',
 '하지만',
 '일때',
 '할때',
 '앞에서',
 '중에서',
 '보는데서',
 '으로써',
 '로써',
 '까지',
 '해야한다',
 '일것이다',
 '반드시',
 '할줄알다',
 '할수있다',
 '할수있어',
 '임에 틀림없다',
 '한다면',
 '등',
 '등등',
 '제',
 '겨우',
 '단지',
 '다만',
 '할뿐',
 '딩동',
 '댕그',
 '대해서',
 '대하여',
 '대하면',
 '훨씬',
 '얼마나',
 '얼마만큼',
 '얼마큼',
 '남짓',
 '여',
 '얼마간',
 '약간',
 '다소',
 '좀',
 '조

In [134]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf = TfidfVectorizer(stop_words = stopwords_kor)
##### TfidfVectorizer 객체 tfidf 생성

In [135]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:          499180 kB
MemAvailable:    3213336 kB


### **중간 정리 **

**book**(모든 외부 도서 데이터) - 1,244,550개

**- merge_book**(문화데이터와 단행본_도서의 교집합,도서관에 존재하는 도서는 추천할 필요가 없기 때문에 제거) - 20249개

**(이 사이에 book과 best_top10의 교집합 찾아서 overview 살리기)**

point_data를 보면 알겠지만 학과·학번별 베스트 대출 TOP10 데이터들이 외부데이터에 1277개 존재 -> (학과·학번별의 관점으로..) 1277/2337 = → **52%의 정확도**

**(이 사이에 book을 24000개로 축소)**

**+ best_top10**( 교집합 中 학과·학번별 TOP10(코사인유사도 비교시 tfidf_matrix에 존재해야하므로)) - 2336개


**결과값을 최대 27000개로 도출해내야 노트북 내에서 구동 가능**

In [136]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1244550 entries, 0 to 1244549
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ISBN         1244550 non-null  object
 1   title        1244550 non-null  object
 2   description  1244550 non-null  object
 3   overview     1244550 non-null  object
dtypes: object(4)
memory usage: 38.0+ MB


In [137]:
merge_book.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20249 entries, 0 to 35282
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         20249 non-null  object
 1   title        20249 non-null  object
 2   description  20249 non-null  object
 3   overview     20249 non-null  object
dtypes: object(4)
memory usage: 791.0+ KB


In [138]:
best_top10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2337 entries, 0 to 2336
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ISBN    2337 non-null   object
 1   서명      2337 non-null   object
 2   분류코드    2337 non-null   object
dtypes: object(3)
memory usage: 54.9+ KB


In [139]:
point_data_original = pd.merge(book, best_top10)
point_data = point_data_original.drop_duplicates(['ISBN'])  # 1290개
point_data = point_data.dropna(subset = ['overview'])  # 1277개 
point_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1277 entries, 0 to 1289
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         1277 non-null   object
 1   title        1277 non-null   object
 2   description  1277 non-null   object
 3   overview     1277 non-null   object
 4   서명           1277 non-null   object
 5   분류코드         1277 non-null   object
dtypes: object(6)
memory usage: 69.8+ KB


In [140]:
mid = pd.concat([book, merge_book], ignore_index = True)
mid = mid.drop_duplicates(['ISBN'], keep = False)
mid.info()
# 1,244,550 - 20249 = 1,224,301

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1224301 entries, 0 to 1244549
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ISBN         1224301 non-null  object
 1   title        1224301 non-null  object
 2   description  1224301 non-null  object
 3   overview     1224301 non-null  object
dtypes: object(4)
memory usage: 46.7+ MB


In [141]:
mid = mid.loc[0:25000].reset_index(drop=True)
mid.info()
# 메모리 부족으로 인해 표본 124만 -> 25000개로 축소

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24453 entries, 0 to 24452
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         24453 non-null  object
 1   title        24453 non-null  object
 2   description  24453 non-null  object
 3   overview     24453 non-null  object
dtypes: object(4)
memory usage: 764.3+ KB


In [142]:
books = pd.concat([mid, point_data], ignore_index = True)
books = books.drop(['분류코드', '서명'], axis = 1)
books.info()
# 25,429 + 1277 = 26,706

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25730 entries, 0 to 25729
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ISBN         25730 non-null  object
 1   title        25730 non-null  object
 2   description  25730 non-null  object
 3   overview     25730 non-null  object
dtypes: object(4)
memory usage: 804.2+ KB


# **books = 최종 정제된 데이터**

**3. 피쳐 벡터화**

books['overview']를 피쳐 벡터화

In [63]:
import warnings
warnings.filterwarnings(action='ignore')
# books['overview']에는 존재하지 않지만, 직접 작성한 한글 불용어 사전 txt 파일에는 존재하는 단어에 대한 warning발생을 억제 
tfidf_matrix = tfidf.fit_transform(books['overview'])
tfidf_matrix.shape
# 메모리가 감당이 가능하면 1,244,550개의 row에 2,974,472개의 column(단어)이 존재
# 정제된 데이터셋 books로는 26,706개의 row에 205,516개의 column(단어)이 존재

(25730, 199795)

In [64]:
tfidf_matrix
# 메모리가 감당이 가능하면 38,660,920개의 0이 아닌 데이터가 존재
# 정제된 데이터셋 books로는 830,109개의 0이 아닌 데이터가 존재

<25730x199795 sparse matrix of type '<class 'numpy.float64'>'
	with 798905 stored elements in Compressed Sparse Row format>

DKLL_2020['overview']를 피쳐 벡터화

In [65]:
# import warnings
# warnings.filterwarnings(action='ignore')
# # DKLL_2020['overview']에는 존재하지 않지만, 직접 작성한 한글 불용어 사전 txt 파일에는 존재하는 단어에 대한 warning발생을 억제 
# DKLLS_2020 = DKLL_2020
# DKLLS_2020 = DKLLS_2020.reset_index(drop=True)
# DKLL_tfidf_matrix = tfidf.fit_transform(DKLLS_2020['overview'])
# DKLL_tfidf_matrix.shape
# # 1,244,550개의 row에 2974472개의 column(단어)이 존재

In [66]:
# DKLL_tfidf_matrix
# # 38,660,920개의 0이 아닌 데이터가 존재
# # 10000 / 22,994,951
# # 5000 / 19,530,492
# # 100 / 4,764,232

In [67]:
!head -n 3 /proc/meminfo
# 20,000일 경우
# MemTotal:       13298572 kB
# MemFree:          328020 kB
# MemAvailable:   10168584 kB
# 30,000일 경우
# MemTotal:       13298572 kB
# MemFree:          176804 kB
# MemAvailable:    6849360 kB
# books일 경우
# MemTotal:       13298580 kB
# MemFree:         5281432 kB
# MemAvailable:    9696852 kB

MemTotal:       13298580 kB
MemFree:          253248 kB
MemAvailable:   10179828 kB


**4. 단어 유사도 판단을 위해 코사인 유사도 사용(linear_kernel)**

코사인 유사도는 벡터의 크기보다는 방향(각도)가 중요함. 앞에서 우리는 TfidfVectorizer를 통해 정규화된 벡터를 뽑아냈으므로 두 벡터의 내적을 통해서 코사인 유사도를 손쉽게 도출해 낼 수 있다. 이를 위해 Scikit-learn에서 제공하는 코사인 유사도 함수인 linear_kernel(cosine_similarity과 결과는 같지만 처리 속도가 보다 빠르므로 선택) 사용

In [68]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.03192697, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.03192697, ..., 1.        , 0.01506893,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.01506893, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [69]:
!head -n 3 /proc/meminfo
# 20,000일 경우
# MemTotal:       13298572 kB
# MemFree:          328020 kB
# MemAvailable:   10168584 kB
# books일 경우
# MemTotal:       13298580 kB
# MemFree:          209008 kB
# MemAvailable:    4046684 kB

MemTotal:       13298580 kB
MemFree:          155612 kB
MemAvailable:    4945856 kB


In [70]:
cosine_sim.shape

(25730, 25730)

**5. ISBN을 기준으로 인덱싱**

In [71]:
indices = pd.Series(books.index, index = books['ISBN']).drop_duplicates()
indices

ISBN
9788953278974        0
9788953283589        1
9788974572549        2
9788926325117        3
9788953283459        4
                 ...  
9788965421832    25725
9791155063989    25726
9788932034584    25727
9791187632337    25728
9791195640881    25729
Length: 25730, dtype: int64

In [72]:
indices['9788980600526']
# 국어국문학과 2020학번 TOP3인 도서 '국어문법론강의'를 예로..

24823

In [73]:
books.iloc[[24823]]
# 국어문법론강의 - 24823

,ISBN,title,description,overview
24823,9788980600526,국어문법론강의,"국어학 교재. 문법학의 형성, 형태소와 단어, 복합어, 파생어, 품사, 명사와 대명...",국어문법론강의 국어학 교재 문법학의 형성 형태소와 단어 복합어 파생어 품사 명사와 ...


**6. 추천받고 싶은 도서의 ISBN을 입력받으면 코사인 유사도를 통해서 가장 유사도가 높은 상위 10개의 도서를 반환**

In [95]:
def lecommenssu_pick(isbn, cosine_sim = cosine_sim):
  # ISBN을 통해서 전체 데이터 기준, 해당 도서의 index값을 가져오기
  idx = indices[isbn]

  # 코사인 유사도 매트릭스(cosine_sim)에서 idx에 해당하는 데이터를 (idx, 유사도) 형태로 가져오기
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 코사인 유사도를 기준으로 내림차순 정렬
  sim_scores = sorted(sim_scores, key = lambda x: x[1], reverse = True)
 
  # 자기 자신(1)을 제외한 10개의 추천 도서를 슬라이싱 
  sim_scores = sim_scores[1:11]

  # 추천 도서 목록 10개의 인덱스 정보를 추출
  book_indices = [i[0] for i in sim_scores]

  # 인덱스 정보를 통해 도서 제목 추출
  return books['title'].iloc[book_indices]

In [96]:
lecommenssu_pick('9788980600526')

15883                                            형태소와 차자표기
18680                (Longman) vocabulary mentor:외국어 영역 어휘
23022                   (수능 고교 단어) 단어 key로 머릿속에 다 넣기:ivory
16986    기초 프랑스어:발음과 문장=(Le)Fran?ais initial : pronocia...
9560                               (3단어로 말문을 트는) 할리우드 영어표현
21009                           한국의 경매 부자들:법원경매로 부자 되기 노하우
12115    경영학:피터 드러커 관점=Management : Peter F. Drucker`s ...
24667                                           번역의 공격과 수비
13290        (사랑의) 수화교실=Korean sign language for beginners
17824                                    (韓中 中韓) 會計·稅務用語辭典
Name: title, dtype: object

In [143]:
!head -n 3 /proc/meminfo

MemTotal:       13298580 kB
MemFree:          500876 kB
MemAvailable:    3215272 kB
